In [1]:
%load_ext autoreload
%autoreload 2

# Analysing the Jacobian of the parameter landscape scan

In order to characterise the mutational properties of our RNA genetic circuits, we can look at attributes of the analytic landscape generated across a subset of parameters. The goal is to predict the *robustness of any mutation*. Because a mutation is a discrete step in sequence space, that translates to a discrete step in parameter space. Previously, we have scanned over a range of values in the parameter space, which, while continuous, is too large to scan over significantly. The calculation to find the value of an analytic given any set of parameters is also not completely trivial. There is some leeway in the requirement for the parameter landscape tobe continuous, as we are more interested in the relative analytic given a relative set of parameters than getting a particularly accurate value. For example, we may want to know whether a circuit is more sensitive if the interaction strength between node 1 is greater between node 2 than node 3. Being able to generalise in such a way is tractable with a discrete sampling of the landscape.

To summarise, these are the goals we want to achieve with analysis of the Jacobian:
1. Generalising enough to identify interactions that do not have a significant effect and can be abstracted away.

Ultimately, we want to see if we can answer some of the above questions without having to scan across the entire parameter space and just sample some circuits.

# Imports

In [2]:
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import diffrax as dfx

from functools import partial
import os
import sys
import umap
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

jax.config.update('jax_platform_name', 'cpu')

if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)

2023-08-30 14:36:09.219020: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-30 14:36:09.219073: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-08-30 14:36:09.219078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [3]:
from synbio_morpher.srv.parameter_prediction.simulator import make_piecewise_stepcontrol
from synbio_morpher.utils.misc.type_handling import flatten_listlike

# Jacobian

In [4]:

from synbio_morpher.utils.modelling.deterministic import Deterministic, bioreaction_sim_dfx_expanded

from jax import jacfwd

unbound_species = ['RNA_0', 'RNA_1', 'RNA_2']
bound_species = sorted(set(flatten_listlike([['-'.join(sorted([x, y])) for x in unbound_species] for y in unbound_species])))
species = unbound_species + bound_species
signal_species = ['RNA_0']
output_species = ['RNA_1']
s_idxs = [species.index(s) for s in signal_species]
output_idxs = [species.index(s) for s in output_species]
signal_onehot = np.array([1 if s in [species.index(ss) for ss in signal_species] else 0 for s in np.arange(len(species))])
signal_target = 2
    
k = 0.00150958097
N0 = 200
y00 = np.array([[N0, N0, N0, 0, 0, 0, 0, 0, 0]]).astype(np.float32)

t0 = 0
t1 = 100
dt0 = 0.0005555558569638981
dt1_factor = 5
dt1 = dt0 * dt1_factor
max_steps = 16**4 * 10

# Reactions
rates = np.array([[1e-4, 1e-4, 1e1],
                  [1e-4, 1e-6, 1e-4],
                  [1e1, 1e-4, 1e-4]])

inputs = np.array([
    [2, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 2, 0, 0, 0, 0, 0, 0],
])
outputs = np.array([
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
])

# Rates
reverse_rates = rates[np.triu_indices(len(rates))]
forward_rates = np.ones_like(reverse_rates) * k

## Try jax

In [35]:
ts = np.linspace(t0, t1, 500)


def statistics(t, y, args):
    return jnp.mean(y), jnp.std(y)

# bb = partial(bioreaction_sim_dfx_expanded,
#         t0=t0, t1=t1, dt0=dt0,
#         signal=None, signal_onehot=signal_onehot,
#         forward_rates=forward_rates,
#         inputs=inputs,
#         outputs=outputs,
#         solver=dfx.Tsit5(),
#         saveat=dfx.SaveAt(
#             ts=np.linspace(t0, t1, 500)),
#             # subs=dfx.SubSaveAt(ts=ts, fn=statistics)),
#         max_steps=max_steps,
#         stepsize_controller=make_piecewise_stepcontrol(t0=t0, t1=t1, dt0=dt0, dt1=dt1)
#         )


bb = partial(bioreaction_sim_dfx_expanded,
             t0=t0, t1=t1, dt0=dt0,
             signal=None, signal_onehot=signal_onehot,
             forward_rates=forward_rates,
             inputs=inputs,
             outputs=outputs,
             solver=dfx.Tsit5(),
             saveat=dfx.SaveAt(
                 ts=ts),
             stepsize_controller=make_piecewise_stepcontrol(
                 t0=t0, t1=t1, dt0=dt0, dt1=dt1)
             )

# Jbb = jacfwd(bb)

sol = bioreaction_sim_dfx_expanded(y00, t0=t0, t1=t1, dt0=dt0,
             signal=None, signal_onehot=signal_onehot,
             forward_rates=forward_rates,
             inputs=inputs,
             outputs=outputs,
             reverse_rates=reverse_rates)

In [37]:
sol.ts

Array([0.        , 0.00055556, 0.00111111, ...,        inf,        inf,
              inf], dtype=float32, weak_type=True)

# Was the sampling good...?